In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings

from dotenv import load_dotenv

warnings.filterwarnings("ignore")

In [3]:
from app.utils.data_loader import DataLoader
from app.utils.health_score import HealthScore
from app.utils.portfolios import Portfolios
from app.utils.portfolios_repo import PortfoliosRepository
from app.utils.portfolios_service import PortfolioService
from app.utils.portprop_matrices import PortpropMatrices
from app.utils.portprop_matrices_repo import PortpropMatricesRepository
from app.utils.rebalancer import Rebalancer
from app.utils.rebalancer_repo import RebalancerRepository

## Load Data

In [4]:
load_dotenv()  # Load environment variables from .env file
print(os.getenv('LOAD_DATA_FROM_DWH'))

false


In [5]:
data_loader=DataLoader(load_from_db=False)
ppm_repo = PortpropMatricesRepository(data_loader=data_loader)
ports_repo = PortfoliosRepository(data_loader=data_loader)
rebalancer_repo = RebalancerRepository(data_loader=data_loader)

In [6]:
client_out_enriched = ports_repo.load_client_out_product_enriched(
    as_of_date="2025-08-31"
)
client_styles = ports_repo.load_client_style(
    as_of_date="2025-08-31"
)

In [7]:
ports_ref_table = {
            'product_mapping' : ports_repo.load_product_mapping(),
            'product_underlying' : ports_repo.load_product_underlying(),
}

In [8]:
ppm_ref_dict = {
    "portprop_factsheet": ppm_repo.load_portprop_factsheet(),
    "portprop_benchmark": ppm_repo.load_portprop_benchmark(),
    "portprop_ge_mapping": ppm_repo.load_portprop_ge_mapping(),
    "portprop_fallback": ppm_repo.load_portprop_fallback(),
    "portprop_ret_eow": ppm_repo.load_portprop_ret_eow(),
    "advisory_health_score": ppm_repo.load_advisory_health_score(),
}

In [9]:
rb_ref_dict = {
    "es_sell_list": rebalancer_repo.load_es_sell_list(),
    "product_recommendation_rank_raw": rebalancer_repo.load_product_recommendation_rank_raw(),
    "mandate_allocation": rebalancer_repo.load_mandate_candidates(),
}

## Instances

In [ ]:
## Portsfolios
ports_all = Portfolios()
ports_all.set_ref_tables(ports_ref_table)
df_out, df_style, port_ids, port_id_mapping = ports_all.create_portfolio_id(client_out_enriched, client_styles, column_mapping=['as_of_date', 'customer_id'])
ports_all.set_portfolio(df_out, df_style, port_ids, port_id_mapping)

## Portfolio Service
port_service = PortfolioService(ports_all)

## Portprop Matrices
ppm = PortpropMatrices(ppm_ref_dict)

## Health Score
hs = HealthScore()

## Rebalancer
rb = Rebalancer(client_investment_style='Moderate High Risk',
                client_classification='UI',
                discretionary_acceptance=0.2,
                new_money=1_000_000,
                product_whitelist=["KKP", "PTTEP"],
                product_blacklist=["KKP GNP", "K-GSELECTU-A(A)"],)
rb.set_ref_tables(rb_ref_dict)

## Portfolio (Service)

In [11]:
## get list of all customer ids
print(port_service.get_all_customer_ids())

[27766, 28359, 23349, 36170, 27144, 35258, 45701, 51637, 61749, 58055, 72106, 76048, 76011, 103542, 52747, 14154, 21427, 12346, 48844, 26925, 52903, 28587, 30084, 17739, 10052, 31843, 80627, 23356, 29901, 12001, 30961, 16906, 109086, 28999, 53015, 35059, 12594, 33624, 18051, 28508, 23791, 28624, 24845, 33713, 37403, 51513, 57478, 79790, 80115, 90529, 93204, 88598, 102165, 103356, 110989, 21373, 21228, 14204, 22556, 22207, 26339, 77908, 26184, 24760, 103969, 31493, 33416, 31626, 22997, 13634, 25735, 13418, 84563, 24268, 15220, 32142, 22242, 28760, 24563, 33182, 18218, 55606, 35978, 58697, 50554, 71436, 65866, 76511, 83965, 86699, 82401, 28464, 21676, 42063, 39626, 45460, 30730, 21053, 111409, 28849, 35981, 37006, 30586, 19673, 24750, 22016, 21050, 35510, 27095, 21512, 22731, 26254, 38286, 43294, 54953, 67508, 59717, 12969, 92990, 103276, 38105, 104313, 108539, 32492, 55566, 49616, 53643, 30676, 35550, 33975, 42032, 12507, 25840, 21974, 36098, 14260, 18771, 25408, 27126, 15021, 39886, 36

In [12]:
## get single port from customer id
port = port_service.get_client_portfolio(customer_id=21105)

In [13]:
## get port outstanding
port.df_out

,customer_id,as_of_date,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,...,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,value,port_id,asset_class_code,weight
240004,21105,2025-08-31,S00081385,CIMBT,TRADE,L,THB,CIMBT,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,66908.95,4965,AA_LE,0.018779
240441,21105,2025-08-31,S00080320,TTA,TRADE,L,THB,TTA,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,30528.00,4965,AA_LE,0.008568
240443,21105,2025-08-31,S00083319,DCC,TRADE,L,THB,DCC,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,238000.00,4965,AA_LE,0.066799
240928,21105,2025-08-31,S00147416,KKP-W6,TRADE,L,THB,KKP-W6,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,626.25,4965,AA_LE,0.000176
241107,21105,2025-08-31,C00058902,SS_2002291655,TRADE,L,THB,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,...,True,0.048,False,<NA>,Not Top-Pick,<NA>,261334.87,4965,AA_CASH,0.073349
241902,21105,2025-08-31,S00080094,BBL,TRADE,L,THB,BBL,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,183490.00,4965,AA_LE,0.051500
241907,21105,2025-08-31,S00080158,TTB,TRADE,L,THB,TTB,Listed Securities,Local Equity,...,True,0.042,False,<NA>,Not Top-Pick,<NA>,48227.70,4965,AA_LE,0.013536
242140,21105,2025-08-31,S00080422,ADVANC,TRADE,L,THB,ADVANC,Listed Securities,Local Equity,...,True,0.042,False,<NA>,Not Top-Pick,<NA>,1293600.00,4965,AA_LE,0.363074
243469,21105,2025-08-31,S00088553,PTTEP,TRADE,L,THB,PTTEP,Listed Securities,Local Equity,...,False,0.042,False,Switch to TOP/SPRC,Not Top-Pick,<NA>,324800.00,4965,AA_LE,0.091162
243714,21105,2025-08-31,S00081361,PSH,TRADE,L,THB,PSH,Listed Securities,Local Equity,...,False,0.042,False,<NA>,Not Top-Pick,<NA>,41000.00,4965,AA_LE,0.011507


In [14]:
port.product_mapping.columns

Index(['product_id', 'src_sharecodes', 'desk', 'port_type', 'currency',
       'product_display_name', 'product_type_desc', 'asset_class_name',
       'symbol', 'pp_asset_sub_class', 'is_risky_asset', 'coverage_prdtype',
       'is_coverage', 'expected_return', 'es_core_port', 'es_sell_list',
       'flag_top_pick', 'flag_tax_saving'],
      dtype='object')

In [15]:
# get port allocation lookthrough
port.get_portfolio_asset_allocation_lookthrough(ppm)

asset_class,port_id,aa_alt,aa_cash,aa_fi,aa_ge,aa_le
0,4965,0.042782,0.091594,0,0,0.865624


In [16]:
# get model allocation
port.get_model_asset_allocation_lookthrough(ppm)

,port_id,port_investment_style,portpop_styles,aa_alt_model,aa_cash_model,aa_fi_model,aa_ge_model,aa_le_model
0,4965,Aggressive Growth,Aggressive,0.105,0.055,0.08,0.684,0.076


In [17]:
port.df_style

,port_id,port_investment_style,portpop_styles
42144,4965,Aggressive Growth,Aggressive


## Healthscore

In [18]:
## get client health score
health_score, health_score_comp = port.get_portfolio_health_score(ppm,hs)

In [19]:
health_score['health_score'].values[0]

np.float64(4.0)

In [20]:
health_score_comp

,port_id,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,value,...,ge_other,expected_return,volatility,is_bulk_risk,underlying_company,issure_risk_group,coverage_prdtype,score_non_cover_global_stock,score_non_cover_local_stock,score_non_cover_mutual_fund
0,4965,S00081385,CIMBT,TRADE,L,THB,CIMBT,Listed Securities,Local Equity,66908.95,...,NaN,7.887314e-04,3.427413e-03,False,CIMBT,NaN,LOCAL_STOCK,0,-1,0
1,4965,S00080320,TTA,TRADE,L,THB,TTA,Listed Securities,Local Equity,30528.00,...,NaN,3.598680e-04,1.563798e-03,False,TTA,NaN,LOCAL_STOCK,0,-1,0
2,4965,S00083319,DCC,TRADE,L,THB,DCC,Listed Securities,Local Equity,238000.00,...,NaN,2.805575e-03,1.219156e-02,False,DCC,NaN,LOCAL_STOCK,0,-1,0
3,4965,S00147416,KKP-W6,TRADE,L,THB,KKP-W6,Listed Securities,Local Equity,626.25,...,NaN,7.382317e-06,3.207968e-05,False,KKP,NaN,LOCAL_STOCK,0,-1,0
4,4965,C00058902,SS_2002291655,TRADE,L,THB,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,261334.87,...,NaN,3.520742e-03,-7.470270e-07,False,<NA>,NaN,N/A,0,0,0
5,4965,S00080094,BBL,TRADE,L,THB,BBL,Listed Securities,Local Equity,183490.00,...,NaN,2.163004e-03,9.399282e-03,False,BBL,NaN,LOCAL_STOCK,0,-1,0
6,4965,S00080158,TTB,TRADE,L,THB,TTB,Listed Securities,Local Equity,48227.70,...,NaN,5.685144e-04,2.470466e-03,False,TTB,NaN,LOCAL_STOCK,0,0,0
7,4965,S00080422,ADVANC,TRADE,L,THB,ADVANC,Listed Securities,Local Equity,1293600.00,...,NaN,1.524913e-02,6.626471e-02,True,ADVANC,1.0,LOCAL_STOCK,0,0,0
8,4965,S00088553,PTTEP,TRADE,L,THB,PTTEP,Listed Securities,Local Equity,324800.00,...,NaN,3.828785e-03,1.663789e-02,False,PTTEP,NaN,LOCAL_STOCK,0,-1,0
9,4965,S00081361,PSH,TRADE,L,THB,PSH,Listed Securities,Local Equity,41000.00,...,NaN,4.833133e-04,2.100227e-03,False,PSH,NaN,LOCAL_STOCK,0,-1,0


In [21]:
health_score_comp.columns

Index(['port_id', 'product_id', 'src_sharecodes', 'desk', 'port_type',
       'currency', 'product_display_name', 'product_type_desc',
       'asset_class_name', 'value', 'weight', 'aa_alt', 'aa_cash', 'aa_fi',
       'aa_ge', 'aa_le', 'ge_em', 'ge_eur', 'ge_jp', 'ge_us', 'ge_other',
       'expected_return', 'volatility', 'is_bulk_risk', 'underlying_company',
       'issure_risk_group', 'coverage_prdtype', 'score_non_cover_global_stock',
       'score_non_cover_local_stock', 'score_non_cover_mutual_fund'],
      dtype='object')

## Rebalancer

In [22]:
new_port, recommendations = rb.rebalance(port, ppm, hs)

In [23]:
recommendations

,transaction_no,batch_no,port_id,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,value,weight,flag,expected_weight,action,amount
0,1,1,4965,S00080094,BBL,TRADE,L,THB,BBL,Listed Securities,Local Equity,183490.0,0.051500,not_monitored_product,0.00,sell,-183490.0000
1,2,1,4965,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,183490.0000
2,3,2,4965,S00083319,DCC,TRADE,L,THB,DCC,Listed Securities,Local Equity,238000.0,0.066799,not_monitored_product,0.00,sell,-238000.0000
3,4,2,4965,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,238000.0000
4,5,3,4965,S00088553,PTTEP,TRADE,L,THB,PTTEP,Listed Securities,Local Equity,324800.0,0.091162,"not_monitored_product, sell_list",0.00,sell,-324800.0000
5,6,3,4965,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,324800.0000
6,7,4,4965,S00088794,SCCC,TRADE,L,THB,SCCC,Listed Securities,Local Equity,430650.0,0.120870,not_monitored_product,0.00,sell,-430650.0000
7,8,4,4965,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,430650.0000
8,9,5,4965,S00080422,ADVANC,TRADE,L,THB,ADVANC,Listed Securities,Local Equity,1293600.0,0.363074,"bulk_risk, issuer_risk",0.19,sell,-616647.8657
9,10,5,4965,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,616647.8657


In [24]:
new_port.df_out

,port_id,product_id,src_sharecodes,desk,port_type,currency,value,product_display_name,product_type_desc,asset_class_name,...,is_risky_asset,coverage_prdtype,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,asset_class_code,weight
0,4965,C00004343,Equity Collateral,TRADE,L,THB,65006.1000,Equity Collateral,Cash,Cash and Cash Equivalent,...,False,N/A,True,0.04800,False,<NA>,Not Top-Pick,<NA>,AA_CASH,0.018245
1,4965,C00058902,SS_2002291655,TRADE,L,THB,261334.8700,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,...,False,N/A,True,0.04800,False,<NA>,Not Top-Pick,<NA>,AA_CASH,0.073349
2,4965,CTHB00000000,Cash Proxy THB,TRADE,L,THB,12134.8807,Cash Proxy THB,Cash,Cash and Cash Equivalent,...,False,N/A,True,0.04800,False,<NA>,Not Top-Pick,<NA>,AA_CASH,0.003406
3,4965,M00073326,KKP CorePath Balanced,TRADE,L,THB,712581.1940,KKP CorePath Balanced,Mutual Fund,Allocation,...,False,MUTUAL_FUND,True,0.06603,False,<NA>,Not Top-Pick,<NA>,NaN,0.200000
4,4965,M00162119,KKP PGE-UH,TRADE,L,THB,356290.5970,KKP PGE-UH,Mutual Fund,Global Equity,...,True,MUTUAL_FUND,True,0.07700,False,<NA>,Top-Pick,<NA>,AA_GE,0.100000
5,4965,M00223018,K-GSELECTU-A(A),TRADE,L,THB,356290.5970,K-GSELECTU-A(A),Mutual Fund,Global Equity,...,True,MUTUAL_FUND,True,0.08700,False,<NA>,Top-Pick,<NA>,AA_GE,0.100000
6,4965,M00228153,K-GPINUH-A(A),TRADE,L,THB,356290.5970,K-GPINUH-A(A),Mutual Fund,Global Equity,...,True,MUTUAL_FUND,True,0.08700,False,<NA>,Top-Pick,<NA>,AA_GE,0.100000
7,4965,S00080158,TTB,TRADE,L,THB,48227.7000,TTB,Listed Securities,Local Equity,...,True,LOCAL_STOCK,False,0.04200,False,<NA>,Not Top-Pick,<NA>,AA_LE,0.013536
8,4965,S00080320,TTA,TRADE,L,THB,30528.0000,TTA,Listed Securities,Local Equity,...,True,LOCAL_STOCK,False,0.04200,False,<NA>,Not Top-Pick,<NA>,AA_LE,0.008568
9,4965,S00080400,BTS,TRADE,L,THB,10541.7600,BTS,Listed Securities,Local Equity,...,True,LOCAL_STOCK,False,0.04200,False,<NA>,Not Top-Pick,<NA>,AA_LE,0.002959


In [25]:
health_score, health_score_comp = new_port.get_portfolio_health_score(ppm,hs)

In [26]:
health_score['health_score'].values[0]

np.float64(10.0)